### IMPORTS

In [1]:
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
import ipynb.fs.full.misc as misc
import ipynb.fs.full.decision as decision

### SERIALIZE THE YAML CONFIG

In [2]:
config = storage.load_yaml('extra/config.yaml')

### STEP 1: PROCESS RAW DATA TO DATAFRAME

In [4]:
dataframe = processing.create_dataframe(config)

In [5]:
dataframe.head(5)

,open,high,low,close,volume,label
Date_Timestamp,,,,,,
2010-01-01,1.43327,1.43356,1.43207,1.43335,39761.000053,1.43141
2010-01-03,1.43024,1.43359,1.42951,1.43141,3001.600003,1.44244
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.400094,1.43634
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.100067,1.44005
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.800085,1.43155


### STEP 2: ADD FEATURES TO DATAFRAME

In [6]:
dataset = features.add(dataframe, config['features'])

In [7]:
dataset.head(5)

,close,momentum,sd,label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409


### STEP 3: TRAIN MODEL ENSEMBLE

In [8]:
ensemble = [None] * len(config['ensemble']['models'])

In [9]:
# LOOP THROUGH REQUESTED MOEDLS
for index, item in enumerate(config['ensemble']['models']):
    
    # MODEL PROPS
    name = list(item)[0]
    params = item[name]
    
    # GENERATE FORMATTED, SPLIT & NORMALIZED DATA FOR TRAINING
    data = features.split(dataset, name, params)

    # TRAIN THE MODEL
    result = training.start(data, name, params)
    
    # APPEND IT TO THE ensemble
    ensemble[index] = {
        'name': name,
        'result': result,
        'scaler': data['scaler']
    }

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1861 samples, validate on 620 samples
Epoch 1/5
1861/1861 [==============================] - 1s 729us/sample - loss: 0.0280 - val_loss: 0.0089
Epoch 2/5
1861/1861 [==============================] - 1s 416us/sample - loss: 0.0048 - val_loss: 0.0044
Epoch 3/5
1861/1861 [==============================] - 1s 411us/sample - loss: 0.0039 - val_loss: 0.0046
Epoch 4/5
1861/1861 [==============================] - 1s 412us/sample - loss: 0.0030 - val_loss: 0.0011
Epoch 5/5
1861/1861 [==============================] - 1s 411us/sample - loss: 0.0024 - val_loss: 0.0017
Train on 1861 samples, validate on 620 samples
Epoch 1/5
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0276 - val_loss: 0.0036
Epoch 2/5
1861/1861 [==============================] - 1s 771us/sample - loss: 0.0032 - val_loss: 7.399

### STEP 4: STITCH ALL REGRESSION PREDICTIONS TOGETHER

In [10]:
regression_predictions = misc.regression_table({
    'dataset': dataset,
    'ensemble': ensemble
})

In [11]:
regression_predictions.head(5)

,label,ensemble,linreg_0,lstm_1,lstm_2
Date_Timestamp,,,,,
2018-01-07,0.341122,0.361590,0.369891,0.366481,0.348398
2018-01-08,0.331619,0.343153,0.354823,0.346063,0.328572
2018-01-09,0.337909,0.331008,0.345550,0.332431,0.315042
2018-01-10,0.357856,0.340677,0.351705,0.345703,0.324622
2018-01-11,0.393802,0.366472,0.371198,0.376614,0.351605


### STEP 5: VISUALIZE DIFFERENCES

In [12]:
# visualize.differences(regression_table)

### STEP 6: CREATE CLASSIFICATION

In [13]:
foo = decision.classify(regression_predictions)

In [17]:
foo.head(5)

,ensemble,linreg_0,lstm_1,lstm_2,label,united_decision,buy,sell,hold
Date_Timestamp,,,,,,,,,
2018-01-07,0.361590,0.369891,0.366481,0.348398,hold,hold,0,0,1
2018-01-08,0.343153,0.354823,0.346063,0.328572,sell,sell,0,1,0
2018-01-09,0.331008,0.345550,0.332431,0.315042,hold,sell,0,1,0
2018-01-10,0.340677,0.351705,0.345703,0.324622,buy,buy,1,0,0
2018-01-11,0.366472,0.371198,0.376614,0.351605,buy,buy,1,0,0


### STEP X: SAVE ENSEMBLE MODELS, PREDICTIONS & BUILD CONFIG

storage.save_ensemble({
    'config': config,
    'ensemble': ensemble,
    'predictions': analysis
})